# **Installing Pre-requisites for our Analysis**

In [128]:
!pip install pandas
!pip install scikit-learn
!pip install pyarrow

In [129]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from IPython.display import display

# **MODEL BUILDING**

## **(i) MODEL TRAINING**

In [130]:
# Reading the csv file using read_csv function

dataset_for_training = r"C:/Users/edwin victor/git repositories/dsp-edwinvictor-justin/data/train.csv"

In [44]:
Training_dataset = pd.read_csv(dataset_for_training)
display(Training_dataset)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [46]:
X = Training_dataset.drop('SalePrice', axis=1)
y = Training_dataset['SalePrice']


In [47]:
# Splitting the dataset into train and test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=50)

### Analyzing the dataset to get overview using some commonly used pandas functions

In [131]:
display(X_train.head(5))
display(y_train.head(5))

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1353,1354,50,RL,56.0,14720,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,3,2010,WD,Normal
1184,1185,20,RL,50.0,35133,Grvl,NaN,Reg,Lvl,AllPub,...,263,0,NaN,NaN,NaN,0,5,2007,WD,Normal
889,890,20,RL,128.0,12160,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2009,WD,Normal
1031,1032,75,RL,102.0,15863,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,8,2009,WD,Normal
1400,1401,50,RM,50.0,6000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,7,2008,WD,Normal


1353    410000
1184    186700
889     149500
1031    197000
1400    120000
Name: SalePrice, dtype: int64

In [132]:
display(X_train.columns) #to get names of the columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [133]:
X_train.shape()
X_test.shape()
#totally there are 1460 rows and 81 columns

TypeError: 'tuple' object is not callable

In [134]:
display(X_train.shape)
display(X_test.shape)
#totally there are 1460 rows and 81 columns (including target feature in original dataset)

(949, 80)

(511, 80)

In [135]:
display(X_Train.info())

NameError: name 'X_Train' is not defined

In [56]:
display(X_train.info())

<class 'pandas.core.frame.DataFrame'>
Index: 949 entries, 1353 to 109
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             949 non-null    int64  
 1   MSSubClass     949 non-null    int64  
 2   MSZoning       949 non-null    object 
 3   LotFrontage    787 non-null    float64
 4   LotArea        949 non-null    int64  
 5   Street         949 non-null    object 
 6   Alley          64 non-null     object 
 7   LotShape       949 non-null    object 
 8   LandContour    949 non-null    object 
 9   Utilities      949 non-null    object 
 10  LotConfig      949 non-null    object 
 11  LandSlope      949 non-null    object 
 12  Neighborhood   949 non-null    object 
 13  Condition1     949 non-null    object 
 14  Condition2     949 non-null    object 
 15  BldgType       949 non-null    object 
 16  HouseStyle     949 non-null    object 
 17  OverallQual    949 non-null    int64  
 18  OverallCond 

None

In [57]:
display(X_train.isnull('SalePrice').sum())
display(X_test.isnull('SalePrice').sum())
# to get total null values in a column

TypeError: isnull() takes 1 positional argument but 2 were given

In [136]:
display(X_test.isnull().sum())
display(X_train.isnull().sum())

Id                0
MSSubClass        0
MSZoning          0
LotFrontage      97
LotArea           0
                 ..
MiscVal           0
MoSold            0
YrSold            0
SaleType          0
SaleCondition     0
Length: 80, dtype: int64

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      162
LotArea            0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
Length: 80, dtype: int64

### Pre-processing The Training Set

In [61]:
selected_features = ['LotArea', 'GrLivArea', 'Neighborhood', 'HouseStyle'] # 2 continuos and 2 Categorical features
target_feature = ['SalePrice'] #Output column

In [62]:
continuos_datatype_features = ['LotArea', 'GrLivArea']
discrete_datatype_features = ['Neighborhood', 'HouseStyle']

In [137]:
Extracted_Selected_Features = X_train[selected_features]
Extracted_Target_Attribute = y_train[target_feature]
# Extracted_Target_Attribute = Training_dataset[target_feature]

KeyError: "None of [Index(['SalePrice'], dtype='object')] are in the [index]"

In [138]:
Extracted_Features_Training = X_train[selected_features]
# Extracted_Target_Attribute = y_train[target_feature]
# Extracted_Target_Attribute = Training_dataset[target_feature]

In [139]:
Extracted_selected_Features.isnull().sum() #checking if there's any null value

NameError: name 'Extracted_selected_Features' is not defined

In [140]:
Extracted_Features_Training.isnull().sum() #checking if there's any null value

LotArea         0
GrLivArea       0
Neighborhood    0
HouseStyle      0
dtype: int64

In [141]:
Extracted_Features_Testing = X_test[selected_features]

In [142]:
Extracted_Target_Attribute.isnull().sum()

SalePrice    0
dtype: int64

In [143]:
#concatenating selected features together as single dataframe

Training_Features = pd.concat([Extracted_Features_Training,y_train], axis=1)
display(Training_Features)

,LotArea,GrLivArea,Neighborhood,HouseStyle,SalePrice
1353,14720,3238,NoRidge,1.5Fin,410000
1184,35133,1572,Timber,1Story,186700
889,12160,1505,NAmes,1Story,149500
1031,15863,3082,SWISU,2.5Fin,197000
1400,6000,1158,BrkSide,1.5Fin,120000
...,...,...,...,...,...
229,3182,1555,Blmngtn,1Story,192500
70,13651,2223,NAmes,1Story,244000
132,7388,1327,NAmes,1Story,150750
1313,14774,2599,NoRidge,2Story,333168


In [144]:
#to get the count of different values from the categorical data type column

Training_Features[discrete_datatype_features].nunique() 

Neighborhood    25
HouseStyle       8
dtype: int64

### Encoding categorical features in Training set

In [148]:
# performing one hot encoding for the selected discrete(categorical) features 

encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
encoder.fit(Training_Features[discrete_datatype_features])
encoded_categories = encoder.transform(Training_Features[discrete_datatype_features])
encoded_discrete_features_df = pd.DataFrame(encoded_categories, columns=encoder.get_feature_names_out(discrete_datatype_features))
display(encoded_discrete_features_df)

# cross-verifying the dimension(rows x columns) matches with the above unique values [33 unique values hence 33 columns]

TypeError: __init__() got an unexpected keyword argument 'sparse'

In [149]:
# performing one hot encoding for the selected discrete(categorical) features 

encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
encoder.fit(Training_Features[discrete_datatype_features])
encoded_categories = encoder.transform(Training_Features[discrete_datatype_features])
encoded_discrete_features_df = pd.DataFrame(encoded_categories, columns=encoder.get_feature_names_out(discrete_datatype_features))
display(encoded_discrete_features_df)

# cross-verifying the dimension matches with the above unique values [33 unique values hence 33 columns]

TypeError: __init__() got an unexpected keyword argument 'sparse'

In [150]:
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

In [151]:
# performing one hot encoding for the selected discrete(categorical) features 

encoder.fit(Training_Features[discrete_datatype_features])
Training_encoded_categories = encoder.transform(Training_Features[discrete_datatype_features])
encoded_discrete_features_training_df = pd.DataFrame(Training_encoded_categories, columns=encoder.get_feature_names_out(discrete_datatype_features))
display(encoded_discrete_features_training_df)

# cross-verifying the dimension matches with the above unique values [33 unique values hence 33 columns]

,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,...,Neighborhood_Timber,Neighborhood_Veenker,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
944,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
945,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
946,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
947,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


### Scaling continuos features in Training set

In [152]:
scaler = StandardScaler()

In [85]:
scaler.fit(X_train[continuos_datatype_features])
scaled_continuos_features_training_df = scaler.transform(X_train[continuos_datatype_features])
display(scaled_continuos_features_training_df)

NameError: name 'scaler' is not defined

In [154]:
scaler.fit(X_train[continuos_datatype_features])
scaled_continuos_features_training_df = scaler.transform(X_train[continuos_datatype_features])
display(scaled_continuos_features_training_df)

array([[ 0.3533823 ,  3.25206567],
       [ 2.16264301,  0.11841038],
       [ 0.12648242, -0.00761297],
       ...,
       [-0.29647315, -0.34242128],
       [ 0.35816847,  2.05014146],
       [ 0.09023161,  0.63002757]])

In [155]:
training_continuous_features_df = pd.DataFrame(scaled_continuos_features_training_df , columns= continuos_datatype_features)
display(training_continuous_features_df)

,LotArea,GrLivArea
0,0.353382,3.252066
1,2.162643,0.118410
2,0.126482,-0.007613
3,0.454690,2.958638
4,-0.419495,-0.660301
...,...,...
944,-0.669263,0.086434
945,0.258634,1.342906
946,-0.296473,-0.342421
947,0.358168,2.050141


### Concatenating the transformed continuos and discrete features together as a single dataframe

In [101]:
Processed_Training_Df = pd.DataFrame([training_continuous_features_df, encoded_discrete_features_training_df] , axis=1)
display(Processed_Training_Df)

TypeError: __init__() got an unexpected keyword argument 'axis'

In [156]:
Processed_Training_Df = pd.concat([training_continuous_features_df, encoded_discrete_features_training_df] , axis=1)
display(Processed_Training_Df)

,LotArea,GrLivArea,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,...,Neighborhood_Timber,Neighborhood_Veenker,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl
0,0.353382,3.252066,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.162643,0.118410,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.126482,-0.007613,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.454690,2.958638,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,-0.419495,-0.660301,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
944,-0.669263,0.086434,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
945,0.258634,1.342906,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
946,-0.296473,-0.342421,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
947,0.358168,2.050141,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


### Training the Model using the processed training set

In [157]:
#training the model with linear regression
model = LinearRegression() 

In [158]:
model.fit(Processed_Training_Df, y_train)

LinearRegression()

## **(ii) MODEL EVALUATION**

### Pre-processing The Testing Set

In [159]:
Extracted_Features_Testing = X_test[selected_features]
Testing_Features = pd.concat([Extracted_Features_Testing,y_test], axis=1)
display(Testing_Features)

,LotArea,GrLivArea,Neighborhood,HouseStyle,SalePrice
930,8925,1466,Timber,1Story,201000
530,10200,1509,Timber,SLvl,175000
1291,1680,1302,BrDale,2Story,119500
1385,5436,1154,IDOTRR,1.5Fin,125500
305,10386,2000,CollgCr,1Story,305900
...,...,...,...,...,...
105,9375,1992,Somerst,2Story,250000
3,9550,1717,Crawfor,2Story,140000
540,14601,1838,Timber,1Story,315000
714,13517,1479,Sawyer,2Story,130500


In [160]:
#to get the count of different values from the categorical data type column

Testing_Features[discrete_datatype_features].nunique() 

Neighborhood    25
HouseStyle       8
dtype: int64

### Encoding categorical features in Testing set

In [161]:
encoder.fit(Testing_Features[discrete_datatype_features])
Testing_encoded_categories = encoder.transform(Testing_Features[discrete_datatype_features])
encoded_discrete_features_testing_df = pd.DataFrame(Testing_encoded_categories, columns=encoder.get_feature_names_out(discrete_datatype_features))
display(encoded_discrete_features_testing_df)

,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,...,Neighborhood_Timber,Neighborhood_Veenker,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
507,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
509,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


### Scaling continuos features in Testing set

In [123]:
scaler.fit(X_test[continuos_datatype_features])
scaled_continuos_features_testing_df = scaler.transform(X_test[continuos_datatype_features])
display(scalded_continuos_features_testing_df)

NameError: name 'scalded_continuos_features_testing_df' is not defined

In [162]:
scaler.fit(X_test[continuos_datatype_features])
scaled_continuos_features_testing_df = scaler.transform(X_test[continuos_datatype_features])
display(scaled_continuos_features_testing_df)

array([[-0.1721897 , -0.119627  ],
       [ 0.01223133, -0.03582136],
       [-1.22013511, -0.43925785],
       ...,
       [ 0.64880935,  0.6053893 ],
       [ 0.49201531, -0.09429041],
       [-0.41793977,  0.21949353]])

In [163]:
testing_continuous_features_df = pd.DataFrame(scaled_continuos_features_testing_df , columns= continuos_datatype_features)
display(testing_continuous_features_df)

,LotArea,GrLivArea
0,-0.172190,-0.119627
1,0.012231,-0.035821
2,-1.220135,-0.439258
3,-0.676852,-0.727705
4,0.039135,0.921122
...,...,...
506,-0.107100,0.905530
507,-0.081787,0.369564
508,0.648809,0.605389
509,0.492015,-0.094290


### Concatenating the transformed continuos and discrete features together as a single dataframe

In [164]:
Processed_Testing_Df = pd.concat([testing_continuous_features_df, encoded_discrete_features_testing_df] , axis=1)
display(Processed_Testing_Df)

,LotArea,GrLivArea,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,...,Neighborhood_Timber,Neighborhood_Veenker,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl
0,-0.172190,-0.119627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.012231,-0.035821,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,-1.220135,-0.439258,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,-0.676852,-0.727705,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.039135,0.921122,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506,-0.107100,0.905530,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
507,-0.081787,0.369564,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
508,0.648809,0.605389,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
509,0.492015,-0.094290,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [170]:
y_pred = model.predict(Processed_Testing_Df)
y_pred

930     201000
530     175000
1291    119500
1385    125500
305     305900
         ...  
105     250000
3       140000
540     315000
714     130500
820     183000
Name: SalePrice, Length: 511, dtype: int64

### Evaluating our model

In [173]:
Rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred)) #rmse = root mean squared error
print ("Root Mean Squared Error:", Rmsle * 100) #using rmse metric to check how good our model has performed

Root Mean Squared Error: 19.357831076666


## **Storing the processed dataframe as parquet file**

In [39]:
# processed_df.to_parquet('C:/Users/edwin victor/git repositories/dsp-edwinvictor-justin/data/house_price_modeling_processed_df.parquet', index = False)

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

In [40]:
# !pip install pyarrow


   ---------------------------------------- 0.0/25.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.1 MB 991.0 kB/s eta 0:00:26
   ---------------------------------------- 0.1/25.1 MB 787.7 kB/s eta 0:00:32
   ---------------------------------------- 0.2/25.1 MB 1.5 MB/s eta 0:00:17
   ---------------------------------------- 0.3/25.1 MB 1.6 MB/s eta 0:00:16
    --------------------------------------- 0.4/25.1 MB 2.0 MB/s eta 0:00:13
    --------------------------------------- 0.6/25.1 MB 2.2 MB/s eta 0:00:12
   - -------------------------------------- 0.7/25.1 MB 2.3 MB/s eta 0:00:11
   - -------------------------------------- 1.0/25.1 MB 2.8 MB/s eta 0:00:09
   - -------------------------------------- 1.1/25.1 MB 2.9 MB/s eta 0:00:09
   -- ------------------------------------- 1.3/25.1 MB 2.9 MB/s eta 0:00:09
   -- ------------------------------------- 1.5/25.1 MB 3.1 MB/s eta 0:00:08
   -- ------------------------------------- 1.7/25.1 MB 3.2 MB/s eta 0:00:08
  

In [41]:
# actual_processed_df = processed_df.to_parquet('C:/Users/edwin victor/git repositories/dsp-edwinvictor-justin/data/house_price_modeling_processed_df.parquet', index = False)

In [94]:
expected_processed_df = pd.read_parquet('C:/Users/edwin victor/git repositories/dsp-edwinvictor-justin/data/house_price_modeling_processed_df.parquet')
expected_processed_df

,LotArea,GrLivArea,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,...,Neighborhood_Veenker,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,SalePrice
0,-0.207142,0.370333,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,True,False,False,208500
1,-0.091886,-0.482512,False,False,False,False,False,False,False,False,...,True,False,False,True,False,False,False,False,False,181500
2,0.073480,0.515013,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,True,False,False,223500
3,-0.096897,0.383659,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,True,False,False,140000
4,0.375148,1.299326,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,-0.260560,0.250402,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,175000
1456,0.266407,1.061367,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,210000
1457,-0.147810,1.569647,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,True,False,False,266500
1458,-0.080160,-0.832788,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,142125


In [95]:
pd.testing.assert_frame_equal(actual_processed_df, expected_processed_df)

# **MODEL INFERENCE**

In [176]:
# Storing the test dataset in a variable

dataset_for_testing = r"C:/Users/edwin victor/git repositories/dsp-edwinvictor-justin/data/test.csv"

In [180]:
# Reading the dataset

Testing_df = pd.read_csv(dataset_for_testing)
display(Testing_df)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal


In [196]:
#scaling and encoding the features
test_scaled = scaler.transform(Testing_df[continuos_datatype_features])
test_encoded = encoder.transform(Testing_df[discrete_datatype_features])
test_scaled

array([[ 0.21791503, -1.23053909],
       [ 0.60049827, -0.3866357 ],
       [ 0.53728886,  0.19805487],
       ...,
       [ 1.42974203, -0.5912774 ],
       [ 0.04709052, -1.08631542],
       [-0.07064965,  0.92112221]])

In [185]:
test_scaled_df = pd.DataFrame(test_scaled,columns=continuos_datatype_features)
test_encoded_df = pd.DataFrame(test_encoded,columns=discrete_datatype_features)
test_scaled_df

ValueError: Shape of passed values is (1459, 33), indices imply (1459, 2)

In [189]:
#converting the scaled values into data frame
test_scaled_df = pd.DataFrame(test_scaled,columns=continuos_datatype_features)
test_scaled_df 

,LotArea,GrLivArea
0,0.217915,-1.230539
1,0.600498,-0.386636
2,0.537289,0.198055
3,-0.019880,0.149331
4,-0.739194,-0.482135
...,...,...
1454,-1.183106,-0.848541
1455,-1.189181,-0.848541
1456,1.429742,-0.591277
1457,0.047091,-1.086315


In [190]:
#converting encoded values into dataframe

test_encoded_df = pd.DataFrame(test_encoded,columns=encoder.get_feature_names_out[discrete_datatype_features])
test_encoded_df

TypeError: 'method' object is not subscriptable

In [191]:
#converting encoded values into dataframe

test_encoded_df = pd.DataFrame(test_encoded,columns=encoder.get_feature_names_out(discrete_datatype_features))
test_encoded_df

,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,...,Neighborhood_Timber,Neighborhood_Veenker,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1455,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1456,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1457,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [193]:
Transformed_test_df = pd.concat([test_scaled_df,test_encoded_df], axis=1)
Transformed_test_df

,LotArea,GrLivArea,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,...,Neighborhood_Timber,Neighborhood_Veenker,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl
0,0.217915,-1.230539,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.600498,-0.386636,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.537289,0.198055,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,-0.019880,0.149331,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,-0.739194,-0.482135,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,-1.183106,-0.848541,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1455,-1.189181,-0.848541,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1456,1.429742,-0.591277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1457,0.047091,-1.086315,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [194]:
predict_house_price = model.predict(Transformed_test_df)
predict_house_price

array([108263.36369508, 152556.57524383, 185285.93541467, ...,
       157936.73796874, 141068.10794168, 197121.11367985])

In [195]:
result_df = Testing_df.copy() 
result_df['Predicted_House_Sale_Price'] = predict_house_price
display(result_df)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,Predicted_House_Sale_Price
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,108263.363695
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,152556.575244
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,185285.935415
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2010,WD,Normal,180309.262864
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,0,NaN,NaN,NaN,0,1,2010,WD,Normal,257432.170389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2006,WD,Normal,79014.012996
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml,78986.545059
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml,157936.737969
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal,141068.107942
